In [2]:
import pandas as pd

In [28]:
entree = ['01 ', '011', '031', '041', '043', '051', '1', '11','11 ','17', '17 ', '31', '41', '43', '51' ]
sortie = ['02 ', '052', '2', '52']
entree_c = ['03 ', '056', '22', '22 ', '3', '42', '56']
sortie_c = ['04 ', '057', '4', '57'] 
columns = ['trip_id', 'client_id','date',  'O', 'D', 'Ot', 'Dt','mode', 
           'O_station','D_station','ligne', 'unkonwn','O_zones','O_x','O_y','D_zones','D_x','D_y', 'O_mode', 'D_mode']
T = 7200

In [3]:
def get_counts(anything):
    counts = {}
    for a in anything:
        if a in counts:
            counts[a] = counts[a] + 1
        else:
            counts[a] = 1
    return counts


In [27]:
def  trip_prediction(data, n, index,trip_id):
    i = 0
    transfer_id = 0
    trips = pd.DataFrame([], columns=columns)
    start = data.loc[index]     #start would be used in the final trip
    end = data.loc[index+n-1]
    trip_start_time = start['HEUR_VALD']   
    while i+1 < n:
        first = data.loc[index+i]
        second = data.loc[index+i+1]
        first_location = arrets[(arrets['COD_TLB_ARRT_STTN'] == first['COD_TLB_ARRT_STTN'])
                                &(arrets['LIBL_LONG_ARRT_STTN'] == first['Lib_Arret'])]
        second_location = arrets[(arrets['COD_TLB_ARRT_STTN'] == second['COD_TLB_ARRT_STTN'])
                                &(arrets['LIBL_LONG_ARRT_STTN'] == second['Lib_Arret'])]
        if (len(first_location) == 0) or (len(second_location) == 0):
            return pd.DataFrame(), trip_id
        first_zones, first_location_x, first_location_y = first_location.iloc[0][['ZONS_ARRT',
                                                                                  'COOR_X_PHSQ',
                                                                                  'COOR_Y_PHSQ']].get_values()
        second_zones, second_location_x, second_location_y = second_location.iloc[0][['ZONS_ARRT',
                                                                                      'COOR_X_PHSQ', 
                                                                                      'COOR_Y_PHSQ']].get_values()
        
        first_data = [trip_id, first['NUMR_CART'], first['JOUR_EXPL_VALD'], 
                      first['Lib_Arret'], second['Lib_Arret'], first['HEUR_VALD'], second['HEUR_VALD'], first['COD_TLB_RES'], 
                      first['COD_TLB_ARRT_STTN'],second['COD_TLB_ARRT_STTN'],first['COD_STF_LIGN'], 'd', first_zones,
                     first_location_x, first_location_y, second_zones, 
                      second_location_x, second_location_y, first['COD_MOD_VALD'], second['COD_MOD_VALD']]
        trips.loc[transfer_id] = first_data
        if second['COD_NATR_VALD'] in sortie:
            trips.loc[transfer_id, 'unkonwn'] = 'no'
            transfer_id = transfer_id + 1 
            i = i + 2
        else:
            trips.loc[transfer_id, 'unkonwn'] = 'd'
            transfer_id = transfer_id + 1
            i = i + 1
            if (second['HEUR_VALD'] - trip_start_time).seconds > T:
                trip_start_time = second['HEUR_VALD']
                trip_id = trip_id + 1
    if (i+1) == n:  # which means the last trip is a 'E'
        if (end['HEUR_VALD'] -  trip_start_time).seconds > T:
            trip_id = trip_id + 1
        
        start_location =  arrets[(arrets['COD_TLB_ARRT_STTN'] == start['COD_TLB_ARRT_STTN'])
                                 &(arrets['LIBL_LONG_ARRT_STTN'] == start['Lib_Arret'])]
        end_location = arrets[(arrets['COD_TLB_ARRT_STTN'] == end['COD_TLB_ARRT_STTN'])
                              &(arrets['LIBL_LONG_ARRT_STTN'] == end['Lib_Arret'])]
        if (len(start_location)==0) or (len(end_location)==0):
            return pd.DataFrame(),trip_id
        start_zones, start_location_x, start_location_y = start_location.iloc[0][['ZONS_ARRT',
                                                                                  'COOR_X_PHSQ',
                                                                                  'COOR_Y_PHSQ']].get_values()
        
        end_zones, end_location_x, end_location_y = end_location.iloc[0][['ZONS_ARRT',
                                                                          'COOR_X_PHSQ',
                                                                          'COOR_Y_PHSQ']].get_values()
        last_data = [trip_id, end['NUMR_CART'], end['JOUR_EXPL_VALD'], 
                     end['Lib_Arret'], start['Lib_Arret'], end['HEUR_VALD'], '', 
                     end['COD_TLB_RES'], end['COD_TLB_ARRT_STTN'],start['COD_TLB_ARRT_STTN'], 
                     end['COD_STF_LIGN'], 'd', end_zones,end_location_x,end_location_y, 
                    start_zones,start_location_x,start_location_y, end['COD_MOD_VALD'], start['COD_MOD_VALD']]
        trips.loc[transfer_id] = last_data
        trips.loc[transfer_id, 'unkonwn'] = 'dh'
        if (end['HEUR_VALD'] - start['HEUR_VALD']).seconds < T:
            return pd.DataFrame(), trip_id
    return trips,trip_id

In [11]:
def  output(data, n):
    trip_id = 0
    trips = pd.DataFrame()
    for index in range(int(len(data)/n)):
        a,b = trip_prediction(data, n, index*n, trip_id)
        trips =  trips.append(a, ignore_index = True)
        trip_id = b + 1
    return trips


In [5]:
path_3 =  'F:/Stage/data/AFC/counts/counts_3'
path_1 = 'F:/Stage/data/AFC/counts/counts_1'
path_arret = "F:/Stage/data/AFC/DM arret.txt"
path_2 = 'F:/Stage/data/AFC/counts/counts_2'
path_4 =  'F:/Stage/data/AFC/counts/counts_4'
path_5 =  'F:/Stage/data/AFC/counts/counts_5'
path_6 =  'F:/Stage/data/AFC/counts/counts_6'
path_7 =  'F:/Stage/data/AFC/counts/counts_7'

In [10]:
data_3 = pd.read_csv(path_3, delimiter="\t", header = 0, encoding="latin-1")
data_3['COD_NATR_VALD'] = data_3['COD_NATR_VALD'].astype('str')
data_3['HEUR_VALD'] = pd.to_timedelta(data_3['HEUR_VALD'])
data_3_sort = data_3.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_3_sort = data_3_sort.reset_index(drop = True)

E:\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [43]:
data_4 = pd.read_csv(path_4, delimiter="\t", header = 0, encoding="latin-1")
data_4['COD_NATR_VALD'] = data_4['COD_NATR_VALD'].astype('str')
data_4['HEUR_VALD'] = pd.to_timedelta(data_4['HEUR_VALD'])
data_4_sort = data_4.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_4_sort = data_4_sort.reset_index(drop = True)

E:\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [33]:
data_2 = pd.read_csv(path_2, delimiter="\t", header = 0, encoding="latin-1")
data_2['COD_NATR_VALD'] = data_2['COD_NATR_VALD'].astype('str')
data_2['HEUR_VALD'] = pd.to_timedelta(data_2['HEUR_VALD'])
data_2_sort = data_2.sort_values(['NUMR_CART','HEUR_VALD'], ascending=[False, True])
data_2_sort = data_2_sort.reset_index(drop = True)

E:\python\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (6,9,12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
get_counts(data_3_sort['COD_MOD_VALD'])

{-1: 6, 1: 404013, 2: 393623, 3: 540069, 4: 53865, 6: 1309}

In [9]:
arrets = pd.read_csv(path_arret, delimiter="\t", header= 0, encoding="latin-1")
arrets = arrets[arrets.ZONS_ARRT != 'ND ']
arrets = arrets[arrets.COOR_X_PHSQ != '-1']
arrets = arrets[arrets.COOR_X_PHSQ != '?']
arrets['COOR_X_PHSQ'] = arrets['COOR_X_PHSQ'].apply(lambda x: float((x.split(',')[0])))
arrets['COOR_Y_PHSQ'] = arrets['COOR_Y_PHSQ'].apply(lambda x: float((x.split(',')[0])))

In [49]:
select_data = data_4_sort.iloc[:210872]
select_data

,Unnamed: 0,NUMR_CART,JOUR_EXPL_VALD,HEUR_VALD,Lib_Arret,COD_TLB_TRNS,COD_TLB_RES,COD_TLB_ARRT_STTN,NUMR_BORN_VALD_EQPM,COD_NATR_VALD,COD_MOD_VALD,COD_STF_TRNS,COD_STF_RES,COD_STF_LIGN
0,12974591,99999948308619046905,14/03/2017,09:10:26,LA DEFENSE-GRANDE ARCHE,1,801,0030050170401,215,51,1,810,801,ND
1,4548889,99999948308619046905,14/03/2017,09:17:45,CHARLES DE GAULLE ETOILE,1,801,0030050170101,208,57,3,810,801,ND
2,8828153,99999948308619046905,14/03/2017,18:40:33,CHARLES DE GAULLE ETOILE,3,001,0030030181200,102,31,3,100,110,ND
3,12974673,99999948308619046905,14/03/2017,18:54:50,MONTPARNASSE,2,002,391003,26600101,1,1,800,852,ND
4,12958512,99999834610306124425,14/03/2017,12:22:32,JACOB,3,001,0030010039027,15861,11,2,100,100,39
5,8820001,99999834610306124425,14/03/2017,14:23:02,PALAIS ROYAL - COMEDIE FRANCAISE,3,001,0030010095019,15372,11,2,100,100,95
6,8820089,99999834610306124425,14/03/2017,14:51:46,SAINT-GERMAIN-DES-PRES,3,001,0030010039026,15680,11,2,100,100,39
7,12958598,99999834610306124425,14/03/2017,19:44:18,JACOB,3,001,0030010039027,6942,11,2,100,100,39
8,12568877,99999661677815476720,14/03/2017,09:44:52,PLACE DE BELGIQUE,3,001,0030010275015,3877,11,2,100,100,275
9,4152293,99999661677815476720,14/03/2017,10:10:28,CHARLES DE GAULLE ETOILE,1,801,0030050170101,216,52,1,810,801,ND


In [ ]:
output_file = output(select_data, 4)

In [47]:
output_file

,trip_id,client_id,date,O,D,Ot,Dt,mode,O_station,D_station,ligne,unkonwn,O_zones,O_x,O_y,D_zones,D_x,D_y,O_mode,D_mode
0,0,99999910079519014654,14/03/2017,Solférino,Saint Exupery,08:38:00,0 days 17:11:00,56,180,370,28,d,4,585747.0,123471.0,5,578838.0,121777.0,2,2
1,1,99999910079519014654,14/03/2017,Saint Exupery,Gare SNCF,17:11:00,0 days 07:39:00,56,370,273,28,d,5,578838.0,121777.0,6,637919.0,131583.0,2,2
2,2,99999871953842078691,14/03/2017,Gare SNCF,Coubertin,07:39:00,0 days 08:01:00,67,273,52,12,d,6,637919.0,131583.0,6,638656.0,139839.0,2,2
3,2,99999871953842078691,14/03/2017,Coubertin,Solférino,08:01:00,,67,52,180,12,dh,6,638656.0,139839.0,4,585747.0,123471.0,2,2
4,3,99999834312022823315,14/03/2017,GENTILLY,GARE DU NORD,07:44:37,0 days 08:04:07,802,0030050180201,271007,ND,no,2,600320.0,123859.0,1,601452.0,131164.0,1,1
5,3,99999641142476692661,14/03/2017,MOUTON - DUVERNET,LUXEMBOURG,07:33:08,0 days 16:26:50,001,0030010038003,0030010038039,38,d,1,599470.0,125555.0,1,600271.0,127357.0,2,2
6,4,99999641142476692661,14/03/2017,LUXEMBOURG,GENTILLY,16:26:50,,001,0030010038039,0030050180201,38,dh,1,600271.0,127357.0,2,600320.0,123859.0,2,1
7,5,99999341872786484931,14/03/2017,ECOLE MILITAIRE,OURCQ,00:09:57,0 days 15:23:04,001,0030030100700,0030030220200,ND,d,1,597767.0,128247.0,1,603698.0,131856.0,3,3
8,6,99999341872786484931,14/03/2017,OURCQ,SAINT-JACQUES,15:23:04,0 days 09:20:47,001,0030030220200,0030030150900,ND,d,1,603698.0,131856.0,1,600044.0,125824.0,3,3
9,7,99999319654609556985,14/03/2017,SAINT-JACQUES,NATIONALE,09:20:47,0 days 13:32:48,1,0030030150900,0030030141100,ND,d,1,600044.0,125824.0,1,601933.0,125860.0,3,3


In [48]:
output_path = 'E:/Study/data/4_0.csv'
output_file.to_csv(output_path, header=True,sep="\t", encoding='latin-1')